В работе представлен расчет ___Rolling Retention___ с использованием данных об установках и логах

В конце кода представлен расчет метрики с использованием функции из личного модуля, где на входа подается два df

In [1]:
# импорт библиотек для работы

import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# создание df из csv-файлов

installs = pd.read_csv('les8_installs.csv', index_col = 0).sort_values(by = 'install_date')
logins = pd.read_csv('les8_logins.csv', index_col = 0)

display(installs)
display(logins)

,client_id,install_date
3037,2739543114822951135,2021-01-01
2944,3490824704637873771,2021-01-01
2943,6916724122858110497,2021-01-01
2942,3129921274471558610,2021-01-01
2941,926550100644139460,2021-01-01
...,...,...
789,1362127979312082210,2021-01-30
788,8416153722973627520,2021-01-30
787,1362978427035410694,2021-01-30
793,5951854166351082017,2021-01-30


,client_id,event_timestamp,is_tester,is_youtuber
12,868610407601689670,2021-02-16 07:53:04,0,0
16,2434015632087848193,2021-02-16 07:51:23,0,0
45,2531632462363212905,2021-02-16 07:52:12,0,0
119,3205304356447992721,2021-02-16 07:54:17,0,0
122,7482763148151863115,2021-02-16 07:54:36,0,0
...,...,...,...,...
5844758,374127069099732578,2021-02-15 12:56:25,0,0
5844759,4905473983468376225,2021-02-15 12:57:54,0,0
5844792,110174818069209226,2021-02-15 12:58:05,0,0
5844800,590901015821838791,2021-02-15 12:58:33,0,0


# Обработка данных перед работой

In [3]:
# приводим даты к нужному типу данных

installs['install_date'] = pd.to_datetime(installs['install_date'])
logins['event_timestamp'] = pd.to_datetime(logins['event_timestamp'])

In [4]:
# приводим event_timestamp к формату даты г-м-д, без указания времени ч-м-с

logins['event_timestamp'] = logins['event_timestamp'].dt.to_period('D').dt.start_time

### Дубли

In [5]:
# проверка данных на наличие дублей

check_dubl_installs = installs.duplicated().sum()
check_dubl_logins = logins.duplicated().sum()

display(check_dubl_installs)
display(check_dubl_logins)

0

163655

In [6]:
# удаляем дубли
logins = logins.drop_duplicates().reset_index(drop=True)

### Пропуски

In [7]:
# проверка данных на наличие пропущенных значений 

check_nan_installs = installs.isnull().sum()
check_nan_logins = logins.isnull().sum()

display(check_nan_installs)
display(check_nan_logins)

client_id       0
install_date    0
dtype: int64

client_id          0
event_timestamp    0
is_tester          0
is_youtuber        0
dtype: int64

In [8]:
# мердж таблицы по клиент ID

summary_installs_logins = installs.merge(
    logins,
    on = 'client_id',
    how = 'inner'
)
summary_installs_logins

,client_id,install_date,event_timestamp,is_tester,is_youtuber
0,2739543114822951135,2021-01-01,2021-01-01,0,0
1,2739543114822951135,2021-01-01,2021-01-02,0,0
2,2739543114822951135,2021-01-01,2021-01-26,0,0
3,3490824704637873771,2021-01-01,2021-01-01,0,0
4,6916724122858110497,2021-01-01,2021-01-01,0,0
...,...,...,...,...,...
122518,1362978427035410694,2021-01-30,2021-02-01,0,0
122519,1362978427035410694,2021-01-30,2021-02-02,0,0
122520,1362978427035410694,2021-01-30,2021-02-04,0,0
122521,5951854166351082017,2021-01-30,2021-01-30,0,0


In [9]:
# определение первоначальной даты регистрации и последней даты захода в логах

summary_installs_logins = (summary_installs_logins
                           .rename(columns = {'install_date':'min_date', 'event_timestamp':'max_date'})
                           .groupby('client_id',as_index = False)
                           .agg({'min_date':'min', 'max_date':'max'})
                          )
summary_installs_logins

,client_id,min_date,max_date
0,438136831630685,2021-01-04,2021-01-05
1,486804146468217,2021-01-27,2021-01-27
2,1158821161632327,2021-01-14,2021-01-14
3,1345466703683100,2021-01-08,2021-01-08
4,1388878700069838,2021-01-11,2021-01-13
...,...,...,...
28419,9221945498303309839,2021-01-20,2021-01-31
28420,9222067806632790758,2021-01-08,2021-01-10
28421,9222597237810583871,2021-01-07,2021-01-07
28422,9222670672591347665,2021-01-12,2021-01-12


In [10]:
# расчет life_time

summary_installs_logins['life_time'] = summary_installs_logins['max_date'] - summary_installs_logins['min_date']
summary_installs_logins['life_time'] = summary_installs_logins['life_time']/np.timedelta64(1, 'D')
summary_installs_logins['life_time'] = summary_installs_logins['life_time'].astype('int')


summary_installs_logins

,client_id,min_date,max_date,life_time
0,438136831630685,2021-01-04,2021-01-05,1
1,486804146468217,2021-01-27,2021-01-27,0
2,1158821161632327,2021-01-14,2021-01-14,0
3,1345466703683100,2021-01-08,2021-01-08,0
4,1388878700069838,2021-01-11,2021-01-13,2
...,...,...,...,...
28419,9221945498303309839,2021-01-20,2021-01-31,11
28420,9222067806632790758,2021-01-08,2021-01-10,2
28421,9222597237810583871,2021-01-07,2021-01-07,0
28422,9222670672591347665,2021-01-12,2021-01-12,0


In [11]:
# расчет количества уникальных польхователей для каждого значения life_time

summary_installs_logins['life_time']
count_users = {}

for char in range(summary_installs_logins['life_time'].min(), summary_installs_logins['life_time'].max()+1):
    result = summary_installs_logins[summary_installs_logins['life_time'] >= char]['client_id'].nunique()
    count_users[char] = result
    
count_users        

{0: 28424,
 1: 12604,
 2: 10166,
 3: 8978,
 4: 8165,
 5: 7589,
 6: 7163,
 7: 6797,
 8: 6467,
 9: 6203,
 10: 5996,
 11: 5816,
 12: 5666,
 13: 5516,
 14: 5383,
 15: 5247,
 16: 5135,
 17: 5031,
 18: 4928,
 19: 4857,
 20: 4772,
 21: 4676,
 22: 4570,
 23: 4492,
 24: 4417,
 25: 4352,
 26: 4289,
 27: 4236,
 28: 4185,
 29: 4109,
 30: 4059,
 31: 3998,
 32: 3958,
 33: 3912,
 34: 3854,
 35: 3808,
 36: 3745,
 37: 3702,
 38: 3658,
 39: 3614,
 40: 3569,
 41: 3519,
 42: 3476,
 43: 3426,
 44: 3374,
 45: 3338,
 46: 3292,
 47: 3257,
 48: 3215,
 49: 3172,
 50: 3140,
 51: 3101,
 52: 3069,
 53: 3042,
 54: 3007,
 55: 2974,
 56: 2940,
 57: 2912,
 58: 2862,
 59: 2828,
 60: 2800,
 61: 2764,
 62: 2733,
 63: 2701,
 64: 2668,
 65: 2638,
 66: 2596,
 67: 2557,
 68: 2528,
 69: 2495,
 70: 2458,
 71: 2414,
 72: 2375,
 73: 2340,
 74: 2299,
 75: 2269,
 76: 2229,
 77: 2183,
 78: 2144,
 79: 2103,
 80: 2073,
 81: 2048,
 82: 2007,
 83: 1972,
 84: 1942,
 85: 1902,
 86: 1874,
 87: 1836,
 88: 1797,
 89: 1763,
 90: 1730,
 91: 1

In [12]:
# расчет Rolling Retention

roll_retan_rate = pd.DataFrame(list(count_users.items()), columns=['lifetime', 'count_users'])
roll_retan_rate['RRRT'] = (roll_retan_rate['count_users'] / roll_retan_rate['count_users']
                           .shift()
                           .fillna(roll_retan_rate['count_users'].max()))

roll_retan_rate

,lifetime,count_users,RRRT
0,0,28424,1.000000
1,1,12604,0.443428
2,2,10166,0.806569
3,3,8978,0.883140
4,4,8165,0.909445
...,...,...,...
131,131,80,0.784314
132,132,58,0.725000
133,133,42,0.724138
134,134,24,0.571429


# используем функцию из модуля RRT_modul.py

In [13]:
# импорт всех функций из личного модуля

from rrt_modul import *

In [14]:
# расчет Rolling Retention путем использования функции, с передачей в нее двух df

calculate_rolling_ret(installs, logins)

,lifetime,count_users,RRRT
0,0,28424,1.000000
1,1,12604,0.443428
2,2,10166,0.806569
3,3,8978,0.883140
4,4,8165,0.909445
...,...,...,...
131,131,80,0.784314
132,132,58,0.725000
133,133,42,0.724138
134,134,24,0.571429
